# quizify

A self service study platform powered by Generative AI, using Hugging face Transformers.

## Introduction

The ` Self-Service Learning platform` is an innovative app designed to revolutionize your learning experience. With this powerful tool, users can effortlessly generate custom quizzes based on any PDF file they download from the web. Gone are the days of tedious manual summarization and translation! Our app leverages the cutting-edge capabilities of Hugging Face transformers to simplify the entire process. Once you've obtained a PDF, simply import it into the app and watch as the magic unfolds.

This learning platform empowers you to summarize the document with ease, condensing its key points into a concise format. Not only that, but our app also offers built-in translation functionality, allowing you to understand the content in your preferred language. But the true power of our app lies in its ability to transform your summarized and translated document into an interactive question-answer service.

By analyzing the text and extracting relevant information, the app generates thought-provoking questions that test your understanding of the material. Whether you're a student striving for academic excellence or a professional looking to enhance your knowledge, the Self Service Quiz Generator platform is your go-to tool for efficient and engaging learning.

## Install the requirements

In [6]:
!pip3 install -r requirements.txt

## Login to hugging face

In [7]:
#Setup
from huggingface_hub import notebook_login
notebook_login()

## Set up Agent and custom tools


In [8]:
from huggingface_hub import login
from transformers import HfAgent
import pandas as pd
import streamlit as st
import downloader as downloader
import readPDF as fileReader
import bertSummarizer_tool as summarizer
import question_generator as quizzer
import translator_tool as translator
import audio_generator as speaker

tools = [
    downloader.download_file_tool,
    fileReader.read_file_tool,
    summarizer.summarizer_tool,
    translator.translate_text_tool,
    speaker.text_to_speech_tool,
]
agent = HfAgent(
    "https://api-inference.huggingface.co/models/bigcode/starcoder",
    additional_tools=tools,
)


## Set up helpers

In [9]:
from docx import Document
import IPython
import soundfile as sf

def play_audio(audio):
    sf.write("speech_converted.wav", audio.numpy(), samplerate=16000)
    return IPython.display.Audio("speech_converted.wav")

def write_to_file(content, file_name):
    name = file_name.split('.pdf')[-1]
    document = Document()
    document.add_paragraph(content)
    document.save(f'{name}.docx')



## Sub Services

### Downloader, Uploader Service

To begin, you can provide a valid URL for your PDF resource.

If you choose to enter a URL, our advanced Hugging Face Agent will seamlessly download the file for you. This process utilizes a specialized tool known as the `download_file_tool` working silently behind the scenes to retrieve the document.

Currently, our platform exclusively supports PDF files at this stage.
As an alternative, you have the option to upload your PDF file. Our agent will utilize a specialized tool called `read_file_tool` to process and extract the content from the document. The extracted information will be saved for further use within the platform.

In [10]:
url = input("Enter a valid URL:")


In [11]:
print("----url is:", url)

Asking agent to download the PDF file from the URL:

In [12]:
def download(url):
    return agent.run(f"Download file from the web {url}", url=url)

downloaded_file = download(url)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 def download(url):                                                                           │
│   2 │   return agent.run(f"Download file from the web {url}", url=url)                           │
│   3                                                                                              │
│ ❱ 4 downloaded_file = download(url)                                                              │
│   5                                                                                              │
│                                                                                                  │
│ in download:2                                                                                    │
│                                                                                                  │
│   1 def download(url):                                                                           │
│ ❱ 2 │   return agent.run(f"Download file from the web {url}", url=url)                           │
│   3                                                                                              │
│   4 downloaded_file = download(url)                                                              │
│   5                                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.11/site-packages/transformers/tools/agents.py:335 in run                  │
│                                                                                                  │
│   332 │   │   ```                                                                                │
│   333 │   │   """                                                                                │
│   334 │   │   prompt = self.format_prompt(task)                                                  │
│ ❱ 335 │   │   result = self.generate_one(prompt, stop=["Task:"])                                 │
│   336 │   │   explanation, code = clean_code_for_run(result)                                     │
│   337 │   │                                                                                      │
│   338 │   │   self.log(f"==Explanation from the agent==\n{explanation}")                         │
│                                                                                                  │
│ /usr/local/lib/python3.11/site-packages/transformers/tools/agents.py:643 in generate_one         │
│                                                                                                  │
│   640 │   │   │   time.sleep(1)                                                                  │
│   641 │   │   │   return self._generate_one(prompt)                                              │
│   642 │   │   elif response.status_code != 200:                                                  │
│ ❱ 643 │   │   │   raise ValueError(f"Error {response.status_code}: {response.json()}")           │
│   644 │   │                                                                                      │
│   645 │   │   result = response.json()[0]["generated_text"]                                      │
│   646 │   │   # Inference API returns the stop sequence                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Error 400: {'error': "Authorization header is invalid, use 'Bearer API_TOKEN'"}

### Summarization Service

let the fun begin! Get ready to dive into the exciting features of our app. How about downloading a summarization of your uploaded document or web content? Let's embark on this thrilling journey together!"

By clicking the `summarize` button below, the Hugging Face agent will generate a summary of your document. Please note that the summarization model used by the agent is the default tool, so the results may not be perfect. If your document is excessively large, there is a chance it may encounter difficulties or exhibit unexpected behavior while processing.

We haven't forgotten about that! You might be eager to download the summary, so go ahead and click the button to access it.

In [13]:
def read_content(file_name):
    return agent.run(f"Read PDF content from {file_name}")
# get the content of PDF
content = read_content(downloaded_file)

def summarize(file_content):
    return agent.run(f"Summaries the text file content {file_content}",file_content = file_content )
# Summaries the content of PDF
summary = summarize(content)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 def read_content(file_name):                                                                │
│    2 │   return agent.run(f"Read PDF content from {file_name}")                                  │
│    3 # get the content of PDF                                                                    │
│ ❱  4 content = read_content(downloaded_file)                                                     │
│    5                                                                                             │
│    6 def summarize(file_content):                                                                │
│    7 │   return agent.run(f"Summaries the text file content {file_content}",file_content = fi    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'downloaded_file' is not defined


### Translation Service

Hmmm, perhaps English is your second language, just like mine! But don't worry, I've got your back 😉. With this tool, you can select between `Italian`, `French`, and `Spanish` languages. Not only that, but you can also have the summary translated into your preferred language for better understanding.

In [14]:
from ipywidgets import interact

def translate(file_content, target_language):
    return agent.run(f"Translate the text file content {file_content} to {target_language}")

def translate_my_file(which_file):
    interact(translate, target_language=['Italian','Spanish','French'], file_content =which_file)

print(translate_my_file(summary))
print(translate_my_file(content))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 def translate_my_file(which_file):                                                          │
│    7 │   interact(translate, target_language=['Italian','Spanish','French'], file_content =wh    │
│    8                                                                                             │
│ ❱  9 print(translate_my_file(summary))                                                           │
│   10 print(translate_my_file(content))                                                           │
│   11                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'summary' is not defined

### Text-To-Speech Service

In addition to all we mentioned so far, if you are keen to listen to the summary or the original uploaded content, in this platform you will be able to generate audio from the file.

In [15]:
def generate_audio(file_content):
    return agent.run(f"Generate audio for a text file content {file_content}")

#You can generate audio for the original file
audio = generate_audio(summary)
play_audio(audio)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 │   return agent.run(f"Generate audio for a text file content {file_content}")               │
│   3                                                                                              │
│   4 #You can generate audio for the original file                                                │
│ ❱ 5 audio = generate_audio(summary)                                                              │
│   6 play_audio(audio)                                                                            │
│   7                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'summary' is not defined

### Quiz Generator Service

Have you studied the summary carefully? Great! That means you're ready, right? Now, let's ask our Hugging Face agent to generate some engaging multiple-choice questions for you! Get ready to put your knowledge to the test!

You have the freedom to choose the language in which you want to be examined! Simply select your desired language option. Additionally, you can specify the number of questions you would like in your requested exam. Tailor the examination experience according to your preferences!
Once the agent has gathered this information, it will utilize another specialized tool called `quiz_generator_tool` to generate the quiz for you. This tool is specifically designed to create dynamic and engaging quizzes based on your selected preferences. Sit back and let the quiz generation process unfold!

In [16]:

def generate_quiz(file_content):
    return agent.run(f"Generating audio for a text file content {file_content}")

quiz =generate_quiz(summary)
print(quiz)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 def generate_quiz(file_content):                                                             │
│   2 │   return agent.run(f"Generating audio for a text file content {file_content}")             │
│   3                                                                                              │
│ ❱ 4 quiz =generate_quiz(summary)                                                                 │
│   5 print(quiz)                                                                                  │
│   6                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'summary' is not defined